In [130]:
import cv2

def draw_bbox(img, loc, color=(0, 0, 255), thickness=2):
    # draw bounding box
    cv2.line(img, (loc[0], loc[1]), (loc[2], loc[3]), color, thickness)
    cv2.line(img, (loc[2], loc[3]), (loc[4], loc[5]), color, thickness)
    cv2.line(img, (loc[4], loc[5]), (loc[6], loc[7]), color, thickness)
    cv2.line(img, (loc[6], loc[7]), (loc[0], loc[1]), color, thickness)
    return img

def draw_category(img, loc, category, color=(0, 0, 255), thickness=2):
    # draw category small font size
    font = cv2.FONT_HERSHEY_PLAIN
    # judge the location of the category
    if loc[1] < 10:
        cv2.putText(img, category, (loc[0], loc[1]+10), font, 1, color, thickness, cv2.LINE_AA)
    else:
        cv2.putText(img, category, (loc[2], loc[3]), font, 1, color, thickness, cv2.LINE_AA)
    return img

def read_label(label_loc):
    # read label file
    with open(label_loc, 'r') as f:
        lines = f.readlines()

    # imagesource = 去除：前的字符串和空格
    imagesource = lines[0].split(':')[1].strip()
    gsd = lines[1].split(':')[1].strip()
    gsd = float(gsd)

    lines_len = len(lines)
    target_nums = lines_len - 2
    targets = []

    for i in range(target_nums):
        # x1,y1,x2,y2,x3,y3,x4,y4,category,difficulty
        # loc = [x1,y1,x2,y2,x3,y3,x4,y4]
        loc = lines[i+2].split(' ')[:8]
        loc = [int(i) for i in loc]
        category = lines[i+2].split(' ')[8]
        difficulty = lines[i+2].split(' ')[9]
        targets.append([loc, category, difficulty])

    return imagesource, gsd, targets

In [129]:
# read all files
import os
import cv2
import sys
from pathlib import Path
import random

root = os.getcwd()  # YOLOv5 root directory
if str(root) not in sys.path:
    sys.path.append(str(root))  # add ROOT to PATH
ROOT = Path(os.path.relpath(root, Path.cwd()))

img_loc = ROOT / "SRSDD/test/images"
label_loc = ROOT / "SRSDD/test/labels"

img_files = os.listdir(img_loc)
label_files = os.listdir(label_loc)

num = len(img_files)

# random select 1 images
idx = random.randint(0, num-1)
img = cv2.imread(str(img_loc / img_files[idx]))

print(img_files[idx])
# show png file

# read label file by the image name
temp = img_files[idx].split('.png')[0]
print(temp)
label_file = temp + '.txt'
print(label_file)

# read label file
imagesource, gsd, targets = read_label(label_loc / label_file)

# draw bounding box
for target in targets:
    loc = target[0]
    print(loc)
    img = draw_bbox(img, loc)
    img = draw_category(img, loc, target[1])

cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


L568.png
L568
L568.txt
[220, 597, 232, 584, 274, 624, 262, 637]
[252, 646, 263, 633, 308, 670, 297, 683]
[266, 636, 277, 623, 322, 660, 311, 673]
[696, 49, 711, 32, 789, 101, 774, 118]
[521, 8, 523, -9, 592, -2, 590, 15]
[717, 775, 733, 757, 781, 798, 765, 816]
[654, 944, 678, 917, 763, 994, 739, 1021]
[479, 814, 502, 788, 564, 844, 541, 870]
[574, 881, 594, 860, 643, 907, 623, 928]
[927, 920, 943, 902, 991, 943, 975, 961]
[803, 834, 822, 812, 871, 855, 852, 877]
[861, 874, 877, 856, 925, 897, 909, 915]


In [152]:
# read all files
import os
import cv2
import sys
from pathlib import Path
import random

root = os.getcwd()  # YOLOv5 root directory
if str(root) not in sys.path:
    sys.path.append(str(root))  # add ROOT to PATH
ROOT = Path(os.path.relpath(root, Path.cwd()))

test_img_loc = ROOT / "SRSDD/test/images"
test_label_loc = ROOT / "SRSDD/test/labels"

test_img_files = os.listdir(test_img_loc)
test_label_files = os.listdir(test_label_loc)

test_num = len(test_img_files)

train_img_loc = ROOT / "SRSDD/train/images"
train_label_loc = ROOT / "SRSDD/train/labels"

train_img_files = os.listdir(train_img_loc)
train_label_files = os.listdir(train_label_loc)

train_num = len(train_img_files)
print(test_num, train_num)

txtsavepath = ROOT / "data/SRSDD/dataSet"

file_train = open(txtsavepath / 'train.txt', 'w')
file_test = open(txtsavepath / 'test.txt', 'w')

for i in range(train_num):
    file_train.write(str(train_img_loc / train_img_files[i]) + '\n')

for i in range(test_num):
    file_test.write(str(test_img_loc / test_img_files[i]) + '\n')

file_train.close()
file_test.close()

# 计算category数量
category = []
for i in range(train_num):
    temp = train_img_files[i].split('.png')[0]
    label_file = temp + '.txt'
    imagesource, gsd, targets = read_label(train_label_loc / label_file)
    for target in targets:
        category.append(target[1])

category = list(set(category))
# 输出编号和category

# 保存category和编号

file_category = open(txtsavepath / 'category.txt', 'w')
for i in range(len(category)):
    file_category.write(str(i) + ' ' + category[i] + '\n')

file_category.close()

# 生成label文件
# 格式 category编号 x1 y1 x2 y2 x3 y3 x4 y4
# 将格式转换为 category编号 xcenter ycenter width height
# 根据图像尺寸归一化
# 源地址 train_label_loc

# read category and category index
category = []
with open (txtsavepath / 'category.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        category.append(line.split(' ')[1].strip())

# category set as dict
category_dict = {}
for i in range(len(category)):
    category_dict[category[i]] = i

lloc = ROOT / 'SRSDD/train/labels'
for i in range(train_num):
    temp = train_img_files[i].split('.png')[0]
    label_file = temp + '.txt'
    imagesource, gsd, targets = read_label(train_label_loc / label_file)
    temp_img = cv2.imread(str(train_img_loc / train_img_files[i]))
    size = temp_img.shape
    with open(lloc / label_file, 'w') as f:
        for target in targets:
            loc = target[0]
            # category编号
            category = category_dict[target[1]]
            xcenter = (loc[0] + loc[2] + loc[4] + loc[6]) / 4 / size[1]
            ycenter = (loc[1] + loc[3] + loc[5] + loc[7]) / 4 / size[0]
            width = (loc[2] - loc[0] + loc[4] - loc[6]) / 2 / size[1]
            height = (loc[3] - loc[1] + loc[5] - loc[7]) / 2 / size[0]
            f.write(str(category) + ' ' + str(xcenter) + ' ' + str(ycenter) + ' ' + str(width) + ' ' + str(height) + '\n')



134 532


In [153]:
test_lloc = ROOT / 'SRSDD/test/labels'
for i in range(test_num):
    temp = test_img_files[i].split('.png')[0]
    label_file = temp + '.txt'
    imagesource, gsd, targets = read_label(test_label_loc / label_file)
    temp_img = cv2.imread(str(test_img_loc / test_img_files[i]))
    size = temp_img.shape
    with open(test_lloc / label_file, 'w') as f:
        for target in targets:
            loc = target[0]
            # category编号
            category = category_dict[target[1]]
            xcenter = (loc[0] + loc[2] + loc[4] + loc[6]) / 4 / size[1]
            ycenter = (loc[1] + loc[3] + loc[5] + loc[7]) / 4 / size[0]
            width = (loc[2] - loc[0] + loc[4] - loc[6]) / 2 / size[1]
            height = (loc[3] - loc[1] + loc[5] - loc[7]) / 2 / size[0]
            f.write(str(category) + ' ' + str(xcenter) + ' ' + str(ycenter) + ' ' + str(width) + ' ' + str(height) + '\n')


In [150]:
# read category and category index
category = []
with open (txtsavepath / 'category.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        category.append(line.split(' ')[1].strip())

# category set as dict
category_dict = {}
for i in range(len(category)):
    category_dict[category[i]] = i

print(category_dict)
for i in range(train_num):
    temp = train_img_files[i].split('.png')[0]
    label_file = temp + '.txt'
    imagesource, gsd, targets = read_label(train_label_loc / label_file)
    temp_img = cv2.imread(str(train_img_loc / train_img_files[i]))
    size = temp_img.shape
    with open(lloc / label_file, 'w') as f:
        for target in targets:
            loc = target[0]
            # category编号
            category = category_dict[target[1]]
            xcenter = (loc[0] + loc[2] + loc[4] + loc[6]) / 4 / size[1]
            ycenter = (loc[1] + loc[3] + loc[5] + loc[7]) / 4 / size[0]
            width = (loc[2] - loc[0] + loc[4] - loc[6]) / 2 / size[1]
            height = (loc[3] - loc[1] + loc[5] - loc[7]) / 2 / size[0]
            f.write(str(category) + ' ' + str(xcenter) + ' ' + str(ycenter) + ' ' + str(width) + ' ' + str(height) + '\n')


{'LawEnforce': 0, 'Fishing': 1, 'Dredger': 2, 'Cell-Container': 3, 'Container': 4, 'ore-oil': 5}


IndexError: list index out of range

In [189]:
# read file and transfer to num then abs finally store in txt file
import os

def read_abs_store(file):
    with open(file, 'r') as f:
        lines = f.readlines()

    num = len(lines)
    # lines to float
    for i in range(num):
        lines[i] = lines[i].split(' ')[:5]
        lines[i] = [abs(float(j)) for j in lines[i]]


    # store in txt file
    with open(file, 'w') as f:
        for i in range(num):
            for j in range(5):
                # 第一个为int
                if j == 0:
                    f.write(str(int(lines[i][j])) + ' ')
                else:
                    f.write(str(lines[i][j]) + ' ')
            f.write('\n')

    return lines

file_loc = 'SRSDD/test/labels'
labes_files = os.listdir(file_loc)

for file in labes_files:
    read_abs_store(file_loc + '/' + file)



In [200]:
def draw_bbox(img, loc, color=(0, 0, 255), thickness=2):
    # draw bounding box
    # loc = [xc, yc, w, h]
    x1 = int((loc[0] - loc[2] / 2) * img.shape[1])
    y1 = int((loc[1] - loc[3] / 2) * img.shape[0])
    x2 = int((loc[0] + loc[2] / 2) * img.shape[1])
    y2 = int((loc[1] + loc[3] / 2) * img.shape[0])
    cv2.line(img, (x1, y1), (x2, y1), color, thickness)
    cv2.line(img, (x2, y1), (x2, y2), color, thickness)
    cv2.line(img, (x2, y2), (x1, y2), color, thickness)
    cv2.line(img, (x1, y2), (x1, y1), color, thickness)
    return img

loc_img = ROOT / 'SRSDD/train/images'
num = len(train_img_files)
idx = random.randint(0, num-1)
img = cv2.imread(str(loc_img / train_img_files[idx]))
temp = train_img_files[idx].split('.png')[0]
label_file = temp + '.txt'

# read labels
temp2 = read_abs_store(train_label_loc / label_file)
loc = []
for i in range(len(temp2)):
    loc.append(temp2[i][1:])

for i in range(len(loc)):
    img = draw_bbox(img, loc[i])

cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


